<a href="https://colab.research.google.com/github/kalinkabel/Exercicios_Data_Science/blob/main/Analisando_dados_musicas_spotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import svm

In [6]:
dataset = pd.read_csv('/content/drive/MyDrive/data.csv', sep=',')

In [7]:
dataset.head()


,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist
0,0,0.0102,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286,1,Mask Off,Future
1,1,0.1990,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588,1,Redbone,Childish Gambino
2,2,0.0344,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173,1,Xanny Family,Future
3,3,0.6040,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230,1,Master Of None,Beach House
4,4,0.1800,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904,1,Parallel Lines,Junior Boys


In [8]:
#CHECANDO VALORES MISSING VALUES
dataset.isnull().sum()


id                  0
acousticness        0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
speechiness         0
tempo               0
time_signature      0
valence             0
target              0
song_title          0
artist              0
dtype: int64

In [9]:
#Resumo estatísticos da base/ Somente dados numericos
dataset.describe()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target
count,2017.000000,2017.000000,2017.000000,2.017000e+03,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000
mean,1008.000000,0.187590,0.618422,2.463062e+05,0.681577,0.133286,5.342588,0.190844,-7.085624,0.612295,0.092664,121.603272,3.968270,0.496815,0.505702
std,582.402066,0.259989,0.161029,8.198181e+04,0.210273,0.273162,3.648240,0.155453,3.761684,0.487347,0.089931,26.685604,0.255853,0.247195,0.500091
min,0.000000,0.000003,0.122000,1.604200e+04,0.014800,0.000000,0.000000,0.018800,-33.097000,0.000000,0.023100,47.859000,1.000000,0.034800,0.000000
25%,504.000000,0.009630,0.514000,2.000150e+05,0.563000,0.000000,2.000000,0.092300,-8.394000,0.000000,0.037500,100.189000,4.000000,0.295000,0.000000
50%,1008.000000,0.063300,0.631000,2.292610e+05,0.715000,0.000076,6.000000,0.127000,-6.248000,1.000000,0.054900,121.427000,4.000000,0.492000,1.000000
75%,1512.000000,0.265000,0.738000,2.703330e+05,0.846000,0.054000,9.000000,0.247000,-4.746000,1.000000,0.108000,137.849000,4.000000,0.691000,1.000000
max,2016.000000,0.995000,0.984000,1.004627e+06,0.998000,0.976000,11.000000,0.969000,-0.307000,1.000000,0.816000,219.331000,5.000000,0.992000,1.000000


**SEPARA AS CLASSES DOS DADOS**

In [10]:
#Separa os meus dados da coluna target para as classes e apaga do dataset
classes= dataset['target']
dataset.drop('target', axis =1, inplace=True)

In [11]:
dataset.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,song_title,artist
0,0,0.0102,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286,Mask Off,Future
1,1,0.1990,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588,Redbone,Childish Gambino
2,2,0.0344,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173,Xanny Family,Future
3,3,0.6040,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230,Master Of None,Beach House
4,4,0.1800,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904,Parallel Lines,Junior Boys


**PRÉ PROCESAMENTO DE DADOS**

In [12]:
def remove_features(lista_feature):
  for i in lista_feature:
     dataset.drop(i,axis=1, inplace= True)
  return 0

In [13]:
#Remove features
remove_features(['id', 'song_title'])

0

In [14]:
dataset.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,artist
0,0.0102,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286,Future
1,0.1990,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588,Childish Gambino
2,0.0344,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173,Future
3,0.6040,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230,Beach House
4,0.1800,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904,Junior Boys


In [15]:
dataset.info

<bound method DataFrame.info of       acousticness  danceability  duration_ms  energy  instrumentalness  key  \
0          0.01020         0.833       204600   0.434          0.021900    2   
1          0.19900         0.743       326933   0.359          0.006110    1   
2          0.03440         0.838       185707   0.412          0.000234    2   
3          0.60400         0.494       199413   0.338          0.510000    5   
4          0.18000         0.678       392893   0.561          0.512000    5   
...            ...           ...          ...     ...               ...  ...   
2012       0.00106         0.584       274404   0.932          0.002690    1   
2013       0.08770         0.894       182182   0.892          0.001670    1   
2014       0.00857         0.637       207200   0.935          0.003990    0   
2015       0.00164         0.557       185600   0.992          0.677000    1   
2016       0.00281         0.446       204520   0.915          0.000039    9   

      l

LABEL ENCODER
Tecnica para transformar dados categoricos em númericos haja vista que o SVM não trabalha com strings só com numeros

In [16]:
from sklearn.preprocessing import LabelEncoder

In [17]:
enc = LabelEncoder()

In [18]:
#fit_transform metodo que transfotma os valores categoricos em  valoers inteiros unicos da coluna artist
inteiros= enc.fit_transform(dataset['artist'])

In [ ]:
#set uma função do python  que visualiza valores unicos
set(inteiros)

In [20]:
#Cria uma nova colua chamada artista_inteiro
dataset['artist_inteiros']= inteiros

In [21]:
dataset.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,artist,artist_inteiros
0,0.0102,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286,Future,449
1,0.1990,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588,Childish Gambino,222
2,0.0344,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173,Future,449
3,0.6040,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230,Beach House,95
4,0.1800,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904,Junior Boys,628


In [22]:
remove_features(['artist'])

0

In [23]:
dataset.head(20)

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,artist_inteiros
0,0.01020,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286,449
1,0.19900,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588,222
2,0.03440,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173,449
3,0.60400,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230,95
4,0.18000,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904,628
5,0.00479,0.804,251333,0.560,0.000000,8,0.1640,-6.682,1,0.1850,85.023,4.0,0.264,356
6,0.01450,0.739,241400,0.472,0.000007,1,0.2070,-11.204,1,0.1560,80.030,4.0,0.308,356
7,0.02020,0.266,349667,0.348,0.664000,10,0.1600,-11.609,0,0.0371,144.154,4.0,0.393,869
8,0.04810,0.603,202853,0.944,0.000000,11,0.3420,-3.626,0,0.3470,130.035,4.0,0.398,321
9,0.00208,0.836,226840,0.603,0.000000,7,0.5710,-7.792,1,0.2370,99.994,4.0,0.386,516


**PANDAS GET_DUMMIE**S

In [24]:
dataset.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,artist_inteiros
0,0.0102,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286,449
1,0.1990,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588,222
2,0.0344,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173,449
3,0.6040,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230,95
4,0.1800,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904,628


In [26]:
#Aplicando o get_dummies nos dados






















dataset= pd.get_dummies(dataset, columns=['artist_inteiros'], prefix= ['artist_inteiros'])

In [27]:
#Visualizando features geradas
dataset.columns

Index(['acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness',
       ...
       'artist_inteiros_1333', 'artist_inteiros_1334', 'artist_inteiros_1335',
       'artist_inteiros_1336', 'artist_inteiros_1337', 'artist_inteiros_1338',
       'artist_inteiros_1339', 'artist_inteiros_1340', 'artist_inteiros_1341',
       'artist_inteiros_1342'],
      dtype='object', length=1356)

len(dataset.columns)

In [28]:
#visualizando as colunas 
dataset.dtypes

acousticness            float64
danceability            float64
duration_ms               int64
energy                  float64
instrumentalness        float64
                         ...   
artist_inteiros_1338      uint8
artist_inteiros_1339      uint8
artist_inteiros_1340      uint8
artist_inteiros_1341      uint8
artist_inteiros_1342      uint8
Length: 1356, dtype: object

In [29]:
#visualizando misssing valus
dataset.isnull().sum()

acousticness            0
danceability            0
duration_ms             0
energy                  0
instrumentalness        0
                       ..
artist_inteiros_1338    0
artist_inteiros_1339    0
artist_inteiros_1340    0
artist_inteiros_1341    0
artist_inteiros_1342    0
Length: 1356, dtype: int64

# **APLICANDO O SVM USANDO PIPELINE**

In [30]:
#IMPORTSNDO AS BIBLIOTECAD DE PIPELINE E PRE-PROCESSDORES
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [31]:
#Treinando o algoritmo de SVM
#Instancia um objeto chamado classificador SVM
cld = svm.SVC().fit(dataset,classes)

In [ ]:
cld

**CROSS VALIDATION**

In [33]:
#Funçao que retorna a acuracia após fazer uma validação cruzada
def Acuracia(cld,X,y):
    resultados= cross_val_predict(cld, X, y, cv=10)
    return metrics.accuracy_score(y, resultados)

In [34]:
#porcenttagem de classificação do modelo
Acuracia(cld, dataset, classes)

0.5577590480912246

In [35]:
#Criando um Pipeline aplicndo um préprocesador scaler padronizando od dados
pip_1= Pipeline([('scaler',StandardScaler()), ('cld',svm.SVC())])

In [ ]:
pip_1.steps

In [37]:
#Usando a função acuracia usando o Pipeline como classificador
Acuracia(pip_1,dataset,classes)

0.58601883986118

In [38]:
#Criando város Pipelline
pip_2= Pipeline([('min_max_scaler', MinMaxScaler()), ('cld', svm.SVC())])

pip_3= Pipeline([('scaler',StandardScaler()), ('cld',svm.SVC(kernel='rbf'))])

pip_4= Pipeline([('scaler',StandardScaler()), ('cld',svm.SVC(kernel= 'poly'))])

pip_5= Pipeline([('scaler',StandardScaler()), ('cld', svm.SVC(kernel= 'linear'))])

**TESTE COM APENAS LABELENCODER NOS DADOS**

In [39]:
Acuracia(pip_1, dataset,classes)

0.58601883986118

In [40]:
Acuracia(pip_2, dataset, classes)

0.7223599405057015

**TESTANDO OS DESEMPENHOS DOS KERNELS**

In [41]:
Acuracia(pip_3,dataset, classes)


0.58601883986118

In [42]:
Acuracia(pip_4, dataset, classes)


0.5751115518096183

In [43]:
Acuracia(pip_5, dataset, classes)

0.6058502726822013

# **COMBINAÇÕES DE PARÂMETROS**

FAZENDO TUNNING E USANDO GRIDSEACHCV

In [44]:
from sklearn.model_selection import GridSearchCV

In [45]:
lista_C= [0.001, 0.01, 0.1, 1, 10, 100]

In [46]:
lista_gamma= [0.001, 0.01, 0.1, 1, 10, 100]

In [47]:
#Define um dicionario que recebe as listas de parâmentros e valores
parametros_grid= dict(cld__C= lista_C, cld__gamma=lista_gamma)

In [48]:
parametros_grid

{'cld__C': [0.001, 0.01, 0.1, 1, 10, 100],
 'cld__gamma': [0.001, 0.01, 0.1, 1, 10, 100]}

In [49]:
#Objeto Grid recebe parâmetros de Pipeline e configurações de cross_validation
grid= GridSearchCV(pip_3, parametros_grid, cv=10, scoring='accuracy')

In [50]:
grid

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('cld', SVC())]),
             param_grid={'cld__C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'cld__gamma': [0.001, 0.01, 0.1, 1, 10, 100]},
             scoring='accuracy')

In [51]:
grid.fit(dataset,classes)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('cld', SVC())]),
             param_grid={'cld__C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'cld__gamma': [0.001, 0.01, 0.1, 1, 10, 100]},
             scoring='accuracy')

In [52]:
x=7*3**2%4

In [53]:
print(x)

3
